In [1]:
import os
import shutil
import tempfile

import pandas as pd

from helpers.cell_type_naming import nice_to_weirds, weird_to_nice
from helpers.running_csx import DockerJob, Experiment, InputFile

In [2]:
experiments_root = "gs://liulab/csx_experiments/cell_type_grouping"
experiment = Experiment(experiments_root, "normal")

In [3]:
!tree -h {experiment.local_path}/..

/mnt/buckets/liulab/csx_experiments/cell_type_grouping/normal/.. [error opening dir]

0 directories, 0 files


## set up input files

### make version of sc refsample with t cells combined

In [4]:
original_sc_refsample = InputFile(
    "screfsampletirosh.txt", 
    "gs://liulab/csx_example_files/Single_Cell_RNA-Seq_Melanoma_SuppFig_3b-d/scRNA-Seq_reference_melanoma_Tirosh_SuppFig_3b-d.txt")

modified_sc_refsample = InputFile(
    "screfsamplemodified.txt",
    os.path.join(experiments_root, "screfsamplemodified.txt"))


In [5]:
!gsutil cp {original_sc_refsample.source_uri} {modified_sc_refsample.source_uri}

Copying gs://liulab/csx_example_files/Single_Cell_RNA-Seq_Melanoma_SuppFig_3b-d/scRNA-Seq_reference_melanoma_Tirosh_SuppFig_3b-d.txt [Content-Type=text/plain]...
/ [1 files][ 87.8 MiB/ 87.8 MiB]                                                
Operation completed over 1 objects/87.8 MiB.                                     


In [6]:
combined_type = "T"
cell_types_to_combine = ["T CD8", "T CD4"]

# combined_type = "Lymphocytes"
# cell_types_to_combine = ["T CD8", "T CD4", "B", "NK"]


In [7]:
thing = os.path.dirname(modified_sc_refsample.source_local_path)
os.makedirs(thing, exist_ok=True)
!tree -h {thing}

/mnt/buckets/liulab/csx_experiments/cell_type_grouping
└── [ 88M]  screfsamplemodified.txt

0 directories, 1 file


In [8]:
with open(original_sc_refsample.source_local_path, "r") as original:
    header = original.readline()
    for cell_type in cell_types_to_combine:
        for weird_name in nice_to_weirds[cell_type]:
            modified_header = header.replace(weird_name, combined_type)
    with open(modified_sc_refsample.source_local_path, "w") as modified:
        modified.write(modified_header)
        shutil.copyfileobj(original, modified)

## configure args

In [9]:
input_file_args = {
    "mixture": InputFile("mixturestirosh.txt", "gs://liulab/csx_example_files/Single_Cell_RNA-Seq_Melanoma_SuppFig_3b-d/mixture_melanoma_Tirosh_SuppFig_3b-d.txt"),
    "refsample": original_sc_refsample,
#     "refsample": modified_sc_refsample,
}

In [10]:
other_args = {
    "rmbatchBmode" : "TRUE",
    "single_cell": "TRUE",
}

## configure and run cibersortx

In [11]:
with tempfile.TemporaryDirectory() as temp_dir:
    job = DockerJob(temp_dir, input_file_args, other_args)
    for copy_command in job.make_copy_commands():
        !{copy_command}
    !tree -h {job.path}
    docker_command = job.make_docker_command()
    print(docker_command)
    !{docker_command}
    !tree -h {job.path}
    !gsutil -m rsync -r -d {job.path} {experiment.gcs_uri}
    !tree -h {experiment.local_path}

Copying gs://liulab/csx_example_files/Single_Cell_RNA-Seq_Melanoma_SuppFig_3b-d/mixture_melanoma_Tirosh_SuppFig_3b-d.txt...
/ [1 files][  6.0 MiB/  6.0 MiB]                                                
Operation completed over 1 objects/6.0 MiB.                                      
Copying gs://liulab/csx_example_files/Single_Cell_RNA-Seq_Melanoma_SuppFig_3b-d/scRNA-Seq_reference_melanoma_Tirosh_SuppFig_3b-d.txt...
\ [1 files][ 87.8 MiB/ 87.8 MiB]                                                
Operation completed over 1 objects/87.8 MiB.                                     
/tmp/tmpyd62tthl
└── [4.0K]  in
    ├── [6.0M]  mixturestirosh.txt
    └── [ 88M]  screfsampletirosh.txt

1 directory, 2 files
docker run \
    --rm \
    -v /tmp/tmpyd62tthl/in:/src/data \
    -v /tmp/tmpyd62tthl:/src/outdir \
    --user "$(id -u):$(id -g)" \
    cibersortx/fractions:latest \
    --username lyronctk@stanford.edu \
    --token dfeba2c8b9d61daebee5fa87026b8e56 \
    --mixture mixturestirosh.txt 